## Citibike Dataset



In [ ]:
%matplotlib inline
import pandas as pd
import MySQLdb as mdb
import matplotlib 
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])
matplotlib.rcParams['figure.figsize'] = (20,10)

In [ ]:
from sqlalchemy import create_engine

conn_string_read = 'mysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = 'localhost', 
    user = 'root',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4', db='citibike_new')
engine_read = create_engine(conn_string_read)



In [ ]:
query = '''
SELECT station_id AS id, 
       AVG(num_bikes_available) AS available_bikes,
       AVG(num_docks_available) AS available_docks,
       AVG(num_bikes_disabled) AS disabled_bikes,
       AVG(num_docks_disabled) AS disabled_docks,
       AVG(num_bikes_available/(num_bikes_available+num_docks_available+num_bikes_disabled+num_docks_disabled)) AS percent_full, 
       DATE_FORMAT(last_reported, '%%Y-%%m-%%d %%H:00:00') AS communication_time 
FROM citibike_new.Status
WHERE  is_renting=1 AND is_installed=1 AND num_bikes_available+num_docks_available+num_bikes_disabled+num_docks_disabled>0
GROUP BY station_id, communication_time
'''
df = pd.read_sql(query, con=engine_read)

In [ ]:
len(df)

In [ ]:
df.head(10)

In [ ]:
df['communication_time'] = pd.to_datetime(df['communication_time'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
df['total_docks'] =  df.available_bikes + df.available_docks + df.disabled_bikes + df.disabled_docks

In [ ]:
df.dtypes

In [ ]:
df.total_docks = df.total_docks.astype(int)

In [ ]:
df = df[ ['id', 'communication_time', 'available_bikes', 'available_docks', 'disabled_bikes', 'disabled_docks', 'total_docks', 'percent_full' ] ]

In [ ]:
df.sort_values(by = ['id', 'communication_time'], inplace=True)

In [ ]:
df

### Writing a Pandas Dataframe in a MySQL Table

Now we will connect to our MySQL server. We will use the SQLAlchemy library of Python.

If you do not have the library, you need to install it by typing in the shell:

In [ ]:
conn_string_write = 'mysql://{user}:{password}@{host}/?charset=utf8mb4'.format(
    host = 'db.ipeirotis.org', 
    user = 'root',
    password = 'krjniJm6OuIyhEoo',
    encoding = 'utf8mb4')
engine_write = create_engine(conn_string_write)


Once we have connected successfully, we need to create our database:

In [ ]:
# Query to create a database
db_name = 'citibike'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8mb4'".format(db=db_name)

# Create a database
engine_write.execute(create_db_query)
# And lets switch to the database
engine_write.execute("USE {db}".format(db=db_name))

In [ ]:
table_name = 'stations'

df.to_sql(
    name = table_name, 
    schema = db_name, 
    con = engine_write,
    if_exists = 'replace',
    index = False, 
    chunksize = 1000)

In [ ]:
# Once we have the data in the table, we also specify a primary key
# If we had FOREIGN KEYS we can add them in the same way
# add_key_query = 'ALTER TABLE nypd ADD PRIMARY KEY(CMPLNT_NUM)'
# engine.execute(add_key_query)